In [1]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras import initializers
import matplotlib.pyplot as mpl
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
#Get the current working directory
cwd = os.getcwd()
print(cwd)

C:\Users\kayde\OneDrive\Desktop\mannada\RainfallData


In [3]:
#the variables as recorded by the station
variables = [
        'dswr',
        'lftx',
        'mslp',
        'p__f',
        'p__u',
        'p__v',
        'p__z',
        'p_th',
        'p_zh',
        'p5_f',
        'p5_u',
        'p5_v',
        'p5_z',
        'p5th',
        'p5zh',
        'p8_f',
        'p8_u',
        'p8_v',
        'p8_z',
        'p8th',
        'p8zh',
        'p500',
        'p850',
        'pottmp',
        'pr_wtr',
        'prec',
        'r500',
        'r850',
        'rhum',
        'shum',
        'temp',
]

#The three regions
regions = [
        '82.5', #A
        '85.0', #B
        '87.5', #C
]

'''
TAKING IN ALL THE X VARIABLES
'''

#define an empty dictionary
boxes = {}

#iterating over regions
for region in regions:
    
    df_li = pd.DataFrame() # empty dataframe
    
    #iterating over variables
    for var in variables:
        
        path = os.path.join(cwd,"DATA\\BOX_20N_%sE\\ncep_%s.dat"%(region,var)) # Cleverly curated path for automation
        
        with open(path) as file: #open the respected variable file for the region
            
            temp = file.read().splitlines()
            temp = [float(i) for i in temp]
            
        df_li[region + '_' + var] = temp #and for every variable, store that data under header: <coordinate_variable>
        
    boxes[region]=df_li #and assign that dataframe for every region
    
'''
TAKING IN ALL THE Y VALUES
'''

rain = []

for i in range(5): # iterate from 0 to 4 i.e. 5 times
    
    path = os.path.join(cwd,"DATA\\rain%d.dat"%(i+1)) # join cwd to path of rain data
    
    with open(path) as file:
        
        lines = file.read().splitlines()
        lines = [float(i) for i in lines]
        rain.append( lines )  #store in rain list

In [7]:
#time for some data refining and model training

#Trimming BOXES, this is to match with the rain data, day-by-day
for region in regions:
    boxes[region] = boxes[region].iloc[4749:,] # removing 4749 rows from front
    boxes[region] = boxes[region].iloc[:10957,] # keeping only 10957 of the rest

#sample take BOX A and PLACE 1
df = boxes['82.5']
rain1 = rain[0]

In [26]:
class predictionModel:
    '''
    This is a class defining the Prediction Neural Network, Its input filtering,
    '''
    def __init__(self,X,Y,N=1,K=31):
        self.__model = self.__neuralNetwork(N,K)
        self.__K = K
        self.__N = N
        self.__xdata = X
        self.__ydata = Y
    def __neuralNetwork(self,N,K):
        model = Sequential(
            [
                Dense(
                    31*N,
                    input_dim=31*N,
                    activation='sigmoid', #next layer is relu, so it needs normalization
                    kernel_initializer=initializers.GlorotNormal(seed=None), #glorot works good with sigmoid
                    bias_initializer='zeros',
                ),
                Dense(
                    K,
                    activation='relu', #relu, for better regression
                    kernel_initializer=initializers.HeNormal(seed=None), #He Normal goes good with relu
                    bias_initializer='zeros',
                ),
                Dense(
                    1,
                    kernel_initializer='normal',
                    bias_initializer='zeros',
                ),
            ]
        )
        model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy','mse'])
        return model
    def trainNetwork(self):
        xtrain, xtest, ytrain, ytest = train_test_split(self.__xdata,self.__ydata,test_size=0.2)#splitting test-train
        print("Initiating Training Sequence")
        history = self.__model.fit(
            xtrain,
            ytrain,
            epochs = 20,
            batch_size = 5,
        )
        print("\n\nInitiating Testing Sequence")
        metrics = self.__model.evaluate(
            xtest,
            ytest,
            batch_size = 5,
        )
    def evaluateNetwork(self):
        evaluators=[
            ('standardize',StandardScaler()),
            ('mlp',KerasRegressor(
                build_fn =  __neuralNetwork(__N,__K),
                epochs = 10,
                batch_size = 5,
                verbose = 0,
            )),
        ]
        pipeline = Pipeline(evaluators)
        results = cross_val_score(
            pipeline,
            xdata,
            ydata,
            cv = KFold(n_splits)
        )
        print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))
        

In [27]:
ydat = rain1
xdat = df.values.tolist()
rainFallPredictor = predictionModel(xdat,ydat,1,31)
rainFallPredictor.trainNetwork()

Initiating Training Sequence
Epoch 1/20
1753/1753 [==============================] - 2s 1ms/step - loss: 62.9487 - accuracy: 0.2153 - mse: 62.9487
Epoch 2/20
1753/1753 [==============================] - 2s 1ms/step - loss: 57.6805 - accuracy: 0.3603 - mse: 57.6805
Epoch 3/20
1753/1753 [==============================] - 2s 1ms/step - loss: 56.8043 - accuracy: 0.3804 - mse: 56.8043
Epoch 4/20
1753/1753 [==============================] - 2s 1ms/step - loss: 56.3170 - accuracy: 0.3442 - mse: 56.3170
Epoch 5/20
1753/1753 [==============================] - 2s 1ms/step - loss: 55.8627 - accuracy: 0.3851 - mse: 55.8627
Epoch 6/20
1753/1753 [==============================] - 2s 1ms/step - loss: 55.3490 - accuracy: 0.3836 - mse: 55.3490
Epoch 7/20
1753/1753 [==============================] - 2s 1ms/step - loss: 55.2675 - accuracy: 0.3927 - mse: 55.2675
Epoch 8/20
1753/1753 [==============================] - 2s 1ms/step - loss: 55.2751 - accuracy: 0.4042 - mse: 55.2751
Epoch 9/20
1753/1753 [=====